# Working interactively on Azure ML

> NOTE!: This notebook is useful for small to medium exploratory workloads that run well on a standard single machine and are not intended for operationalisation and deployment. Do check out Prompt Flow for full AI solution development, and the job submission capabilities and pipeline orchestration of the Azure ML SDKv2 and CLI v2!

### [Optional] Create a custom interactive python Environent

You can run this notebook in the default AML python 3.10 environment on your compute instance. 

To discover custom environments, Find the button to launch a terminal on the compute instance, and run through [this gist](https://gist.github.com/lindacmsheard/c6df1b511f09a3a5939280bd51ebe6b9) to create a custom conda environment. Refresh the page before coming back to this notebook.

This folder contains a suggested environment conda file - if you use it, you can skip the pip install commands below. 

In [3]:
# verify current kernel
import sys
print(sys.version)
print(sys.executable)


3.10.11 | packaged by conda-forge | (main, May 10 2023, 18:58:44) [GCC 11.3.0]
/home/lisheard/miniforge3/envs/aml_310/bin/python3.10


### Best practice - handling keys
> do not place any API keys or other sensitive data directly into a notebook. Here, we use a .env file and the python-dotenv package - that allows us to gitignore the .env file to keep it from being pushed to your repo.

Use the terminal or the File tree on the left to create a file named `.env`

To support the Azure OpenAI example below, add the following line into the file:
```
AZURE_OPENAI_KEY=<the key of the resource provided for the hack>
```



In [4]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv
load_dotenv('./.env')

assert('AZURE_OPENAI_KEY' in os.environ)

In [19]:
%pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 3.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 2.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 4.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [21]:
#Note: The openai-python library support for Azure OpenAI is in preview.
      #Note: This code sample requires OpenAI Python library version 1.0.0 or higher.
import os
from openai import AzureOpenAI


client = AzureOpenAI(
  azure_endpoint = "https://co-openai-sandbox-01.openai.azure.com/", 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-15-preview"
)

system="""
"You are an AI assistant that helps people produce sample data. 
Do not return additional commentary or markdown notation around the data.
the intention is to use your response programmatically.

Example response:
[
    {"ID": 1, "Category": "Electronics", "Description": "Smartphone with 128GB storage"},
    {"ID": 2, "Category": "Fashion", "Description": "Leather wallet in black"},
    ...
]
"""

prompt="""
Please can you create a dataset with three columns, 
make up any data, 10 rows in total, mix of numeric, categorical and text.
Don't truncate the table, return the table only, in json format
"""

message_text = [{"role":"system","content":system},
                {"role":"user","content":prompt}]

completion = client.chat.completions.create(
  model="gpt-4-0125", # model = "deployment_name"
  messages = message_text,
  temperature=0.7,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)



In [22]:
print(completion.choices[0].message.content)

[
    {"ID": 1, "Category": "Electronics", "Price": 999, "Description": "Latest model smartphone with 256GB storage and 12GB RAM"},
    {"ID": 2, "Category": "Books", "Price": 14, "Description": "Hardcover fantasy novel by a bestselling author"},
    {"ID": 3, "Category": "Fashion", "Price": 75, "Description": "Eco-friendly cotton t-shirt, size M"},
    {"ID": 4, "Category": "Electronics", "Price": 250, "Description": "Wireless noise-cancelling headphones"},
    {"ID": 5, "Category": "Home & Garden", "Price": 35, "Description": "Stainless steel kitchen knife set"},
    {"ID": 6, "Category": "Sports", "Price": 120, "Description": "Men's running shoes, size 11"},
    {"ID": 7, "Category": "Beauty", "Price": 45, "Description": "Vegan skincare set including moisturizer and cleanser"},
    {"ID": 8, "Category": "Toys & Games", "Price": 60, "Description": "Educational STEM building toy set"},
    {"ID": 9, "Category": "Automotive", "Price": 300, "Description": "Premium car detailing kit"},
 

In [25]:
# Warning - this will only work if your prompt ensures that nothing but valid json is returned in the response. Remember this is a model finetuned for chat
import json
data = json.loads(completion.choices[0].message.content)

In [31]:
import pandas as pd
df = pd.DataFrame(data)
df.head()

,ID,Category,Price,Description
0,1,Electronics,999,Latest model smartphone with 256GB storage and...
1,2,Books,14,Hardcover fantasy novel by a bestselling author
2,3,Fashion,75,"Eco-friendly cotton t-shirt, size M"
3,4,Electronics,250,Wireless noise-cancelling headphones
4,5,Home & Garden,35,Stainless steel kitchen knife set


# Upload and register the dataset in Azure ML

Work through [this page](https://learn.microsoft.com/en-us/azure/machine-learning/tutorial-explore-data?view=azureml-api-2) to register your data into the Azure ML workspace

> :bulb: Tip: write the dataframe out locally to a file first. 

> :bulb: **Recommendation: please switch back to the curated python kernel, `Python 3.10 - SDK v2`**

> *To learn more about how to install the Azure ML SDKv2 in your own environment, see [here](https://learn.microsoft.com/en-us/python/api/overview/azure/ml/install?view=azure-ml-py).*